### Router Chain

In [1]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser

In [19]:
load_dotenv()

openai_api_key = os.environ["OPENAI_API_KEY"]
model_name = "gpt-3.5-turbo-0125"
max_tokens = 1000

In [26]:
model_physics = "gpt-4o"
model_math = "gpt-3.5-turbo-0125"
model_history = "gpt-4"

llm_default = ChatOpenAI(
    model=model_name,
    temperature=0.5,
    openai_api_key=openai_api_key,
    max_tokens=max_tokens
)

llm_physics = ChatOpenAI(
    model=model_physics,
    temperature=0.5,
    openai_api_key=openai_api_key,
    max_tokens=max_tokens
)

llm_math = ChatOpenAI(
    model=model_math,
    temperature=0.5,
    openai_api_key=openai_api_key,
    max_tokens=max_tokens
)

llm_history = ChatOpenAI(
    model=model_history,
    temperature=0.5,
    openai_api_key=openai_api_key,
    max_tokens=max_tokens
)

In [21]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""

math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts,
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""

In [22]:
# Defining the prompt templates
prompt_infos = [
    {
        "name": "physics",
        "description": "Good for answering questions about physics",
        "prompt_template": physics_template,
        "llm":llm_physics
    },
    {
        "name": "math",
        "description": "Good for answering math questions",
        "prompt_template": math_template,
        "llm":llm_math
    },
    {
        "name": "History",
        "description": "Good for answering history questions",
        "prompt_template": history_template,
        "llm":llm_history
    }
]

In [23]:
destination_chains = {}

for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    llm_model = p_info["llm"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm_model, prompt=prompt)
    destination_chains[name] = chain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [24]:
print(destinations_str)

physics: Good for answering questions about physics
math: Good for answering math questions
History: Good for answering history questions


In [25]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

<>:1: SyntaxWarning: invalid escape sequence '\ '
<>:1: SyntaxWarning: invalid escape sequence '\ '
C:\Users\Admin\AppData\Local\Temp\ipykernel_11664\2160379643.py:1: SyntaxWarning: invalid escape sequence '\ '
  MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \


In [27]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm_default, prompt=default_prompt)

In [29]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

In [30]:
router_chain = LLMRouterChain.from_llm(llm_default, router_prompt)

In [31]:
chain = MultiPromptChain(router_chain=router_chain,
                         destination_chains=destination_chains,
                         default_chain=default_chain, verbose=True
                         )

In [32]:
chain.run("What is Black Body Radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is Black Body Radiation?'}
> Finished chain.


'Black body radiation refers to the electromagnetic radiation emitted by an idealized object called a "black body," which absorbs all incident radiation, regardless of frequency or angle of incidence. A black body in thermal equilibrium emits radiation with a characteristic spectrum that depends only on its temperature, not on its shape or composition. This spectrum is continuous and peaks at a wavelength inversely proportional to the temperature, a relationship described by Wien\'s displacement law. The total energy emitted across all wavelengths is proportional to the fourth power of the temperature, as given by the Stefan-Boltzmann law. Black body radiation is a key concept in quantum mechanics and thermodynamics, historically significant in the development of quantum theory, as it led to Planck\'s law and the concept of quantized energy levels.'

In [33]:
chain.run("What is 2+2?")



> Entering new MultiPromptChain chain...
math: {'input': 'What is 2+2?'}
> Finished chain.


'2 + 2 = 4'

In [34]:
chain.run("I want to understand probabilistic theories in statistics")



> Entering new MultiPromptChain chain...
physics: {'input': 'I want to understand probabilistic theories in statistics'}
> Finished chain.


"Probabilistic theories in statistics revolve around the use of probability to make inferences or predictions about uncertain events or data. Here's a concise overview to help you understand the basics:\n\n1. **Probability Basics**: Probability is a measure of the likelihood that a certain event will occur. It ranges from 0 (impossible event) to 1 (certain event).\n\n2. **Random Variables**: These are variables whose values result from random phenomena. They can be discrete (e.g., rolling a die) or continuous (e.g., measuring the height of people).\n\n3. **Probability Distributions**: These describe how probabilities are distributed over the values of a random variable. Common examples include the normal distribution, binomial distribution, and Poisson distribution.\n\n4. **Expectation and Variance**: The expectation (or expected value) is the average value of a random variable if you were to repeat an experiment many times. Variance measures how much the values of the random variable 

In [17]:
chain.run("Please tell me where did exactly Napolean Bonaparte die?")



> Entering new MultiPromptChain chain...
History: {'input': 'Please tell me where Napolean Bonaparte died exactly?'}
> Finished chain.


'Napoleon Bonaparte died on the island of Saint Helena in the South Atlantic Ocean on May 5, 1821.'

In [35]:
chain.run("What is your name?")



> Entering new MultiPromptChain chain...
None: {'input': 'What is your name?'}
> Finished chain.


'I am an AI digital assistant and do not have a personal name. You can simply refer to me as Assistant. How can I assist you today?'

In [36]:
chain.run("highlight one incident happened in the field of physics after 2022")



> Entering new MultiPromptChain chain...
physics: {'input': 'Highlight one incident that happened in the field of physics after 2022.'}
> Finished chain.


"I'm sorry, but I don't have access to information or events in the field of physics that occurred after 2022. My training data only goes up to October 2023, and I do not have real-time updates. If you have any other questions about physics or events up to 2022, feel free to ask!"